In [28]:
import sys
sys.path.append('/Users/victoria/Documents/Scripts/Python/DGL-PTM/dgl_ptm')
import torch
import dgl_ptm
import os
os.environ["DGLBACKEND"] = "pytorch"


This version features the desired income generation function; a fitted function is used as a decent placeholder to determine consumption. There are many aspects (such as optimization and personal perception of theta) which will be required in future but are currently not utilized. With the model in its current arrangement, the agents stop trading and stabillize after ~7 time steps. I have included a commented-out section at the bottom which uses matplotlib pyplot if you wish to get a quick look.
The current version of the config.yaml file is not formatted correctly and should not be used as is.

In [29]:
model = dgl_ptm.PovertyTrapModel(model_identifier='testwithnn')

In [30]:
model.set_model_parameters(default=False, **{'number_agents': 200000,'seed':1, 'step_target': 100,    
'capital_dist': {'type':'uniform','parameters':[0.1,10.],'round':False,'decimals':None}, 
 'steering_parameters': {'npath':'./agent_data_25.zarr',
                            'epath':'./edge_data_25', 
                            'ndata':['all_except',['a_table']],
                            'edata':['all'],
                            'mode':'xarray',
                            'wealth_method':'weighted_transfer',
                            'income_method':'income_generation',
                            'tech_gamma': torch.tensor([0.3,0.35,0.45]),
                            'tech_cost': torch.tensor([0,0.15,0.65]),
                            'consume_method':'estimated_bellman_consumption',
                            #'nn_path':'/Users/victoria/Documents/Scripts/Python/DGL-PTM/DGL_testing/nn_data/both_PudgeFiveLayer_1024/0409_175055/model_best.pth',
                            'nn_path':'/Users/victoria/Documents/Scripts/Python/DGL-PTM/DGL_testing/nn_data/both_PudgeSixLayer_1024/0409_190204/model_best.pth',
                            'adapt_m':torch.tensor([0,0.5,0.9]),
                            'adapt_cost':torch.tensor([0,0.25,0.45]),
                            'depreciation': 0.6,
                            'discount': 0.95,
                            'm_theta_dist': {'type':'multinomial','parameters':[[0.01 ,0.1, 0.79, 0.1],[0.1, 0.5, 0.8, 1]],'round':False,'decimals':None},
                            'm_attach_dist': {'type':'uniform','parameters':[0.001,1],'round':False,'decimals':None},
                            'del_prob':0.025,
                            'ratio':0.2,
                            'weight_a':0.69,
                            'weight_b':5, 
                            'truncation_weight':1.0e-10,
                            'step_type': 'custom'}})

#print(model.number_agents)  
#print(model.nn_path)  
print(model.steering_parameters)                        

{'npath': './agent_data_4.zarr', 'epath': './edge_data_4', 'ndata': ['all_except', ['a_table']], 'edata': ['all'], 'mode': 'xarray', 'wealth_method': 'weighted_transfer', 'income_method': 'income_generation', 'tech_gamma': tensor([0.3000, 0.3500, 0.4500]), 'tech_cost': tensor([0.0000, 0.1500, 0.6500]), 'consume_method': 'estimated_bellman_consumption', 'nn_path': '/Users/victoria/Documents/Scripts/Python/DGL-PTM/DGL_testing/nn_data/both_PudgeSixLayer_1024/0409_190204/model_best.pth', 'adapt_m': tensor([0.0000, 0.5000, 0.9000]), 'adapt_cost': tensor([0.0000, 0.2500, 0.4500]), 'depreciation': 0.6, 'discount': 0.95, 'm_theta_dist': {'type': 'multinomial', 'parameters': [[0.01, 0.1, 0.79, 0.1], [0.1, 0.5, 0.8, 1]], 'round': False, 'decimals': None}, 'm_attach_dist': {'type': 'uniform', 'parameters': [0.001, 1], 'round': False, 'decimals': None}, 'del_prob': 0.04, 'ratio': 0.2, 'weight_a': 0.69, 'weight_b': 5, 'truncation_weight': 1e-10, 'step_type': 'custom'}


In [31]:
model.initialize_model()

1 42
Created graph with 200000 agents.


In [34]:
model.run()
#100,000 took about 1.5 min on CPU
#1,000,000 took about 16 min on CPU
#need to tune characteristic distance


performing step 14 of 100


RuntimeError: execution of step failed for step 13

In [ ]:
import dgl
src_ids = torch.tensor([1, 2, 2, 4, 4,0,0])
dst_ids = torch.tensor([4, 3, 0, 0, 1,2,3])
g = dgl.graph((src_ids, dst_ids), num_nodes=5)
print(g.edges(order='eid'))

connecting_nodes=torch.tensor([1,2,3,4,0])
sample = dgl.sampling.sample_neighbors(g, connecting_nodes, 2, edge_dir ="out")
print(sample.edges(order='eid'))

#isolate and delete edges with no partner
print(sample.out_degrees(connecting_nodes))
sample.ndata['out_degree'] = sample.out_degrees()

print(sample.ndata['out_degree'])

print(sample.ndata['out_degree'] == 2)

#def nodes_without_two_neighbors(nodes):
    #return (nodes.data['out_degree'] != 2)

#print(sample.filter_nodes(nodes_without_two_neighbors))
#insufficient_connections=sample.filter_nodes(nodes_without_two_neighbors)

# Edges from nodes with insufficient connections
def edges_to_remove(edges):
    print(f"here:{edges.src['out_degree']}")
    return (edges.src['out_degree'] != 2)

insufficient_connections = sample.filter_edges(edges_to_remove)

print(insufficient_connections)

sample.remove_edges(insufficient_connections)
print(sample.edges(order='eid'))
print(sample.edges(order='eid')[1])

even_indices_tensor = sample.edges(order='eid')[1][::2]
odd_indices_tensor = sample.edges(order='eid')[1][1::2]
existing_connections = g.has_edges_between(even_indices_tensor, odd_indices_tensor)
print(f'even_indices_tensor:{even_indices_tensor}')
print(f'odd_indices_tensor:{odd_indices_tensor}')
print(f'existing_connections:{existing_connections}')
print(~existing_connections)


even_indices_tensor = sample.edges(order='eid')[1][::2][~existing_connections]
odd_indices_tensor = sample.edges(order='eid')[1][1::2][~existing_connections]
prob_tensor = torch.rand(even_indices_tensor.size(0))


print(even_indices_tensor)
print(odd_indices_tensor)
print(prob_tensor)

successful_links = prob_tensor > 0.5
print(successful_links)
#add "new" edges to the original graph

g.add_edges(even_indices_tensor[successful_links], odd_indices_tensor[successful_links])
print(g.edges(order='eid'))

g.add_edges(even_indices_tensor[successful_links], odd_indices_tensor[successful_links])

print(g.edges(order='eid'))

ten=torch.tensor([1,2,3,4,5,6,7,8,9,10])
print(ten.shape)






(tensor([1, 2, 2, 4, 4, 0, 0]), tensor([4, 3, 0, 0, 1, 2, 3]))
(tensor([1, 2, 2, 4, 4, 0, 0]), tensor([4, 0, 3, 0, 1, 3, 2]))
tensor([1, 2, 0, 2, 2])
tensor([2, 1, 2, 0, 2])
tensor([ True, False,  True, False,  True])
here:tensor([1, 2, 2, 2, 2, 2, 2])
tensor([0])
(tensor([2, 2, 4, 4, 0, 0]), tensor([0, 3, 0, 1, 3, 2]))
tensor([0, 3, 0, 1, 3, 2])
even_indices_tensor:tensor([0, 0, 3])
odd_indices_tensor:tensor([3, 1, 2])
existing_connections:tensor([ True, False, False])
tensor([False,  True,  True])
tensor([0, 3])
tensor([1, 2])
tensor([0.2094, 0.7066])
tensor([False,  True])
(tensor([1, 2, 2, 4, 4, 0, 0, 3]), tensor([4, 3, 0, 0, 1, 2, 3, 2]))
(tensor([1, 2, 2, 4, 4, 0, 0, 3, 3]), tensor([4, 3, 0, 0, 1, 2, 3, 2, 2]))
torch.Size([10])


In [ ]:
import dgl.sparse as spmatrix

src_ids = torch.tensor([1, 4, 2, 3, 2, 0, 4, 0, 0])
dst_ids = torch.tensor([4, 1, 3, 2, 0, 2, 0, 4, 3])
g = dgl.graph((src_ids, dst_ids), num_nodes=10)
print(g.edges(order='eid'))


adjacency_matrix = g.adj()
neighbor_sum = spmatrix.sum(adjacency_matrix, dim=1)

print(neighbor_sum)


(tensor([1, 4, 2, 3, 2, 0, 4, 0, 0]), tensor([4, 1, 3, 2, 0, 2, 0, 4, 3]))
tensor([3., 1., 2., 1., 2., 0., 0., 0., 0., 0.])


In [ ]:
'''
# try to see something happening
import zarr
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.animation as animation



agent_data = zarr.open('/Users/victoria/Documents/Scripts/Python/DGL-PTM/DGL_testing/agent_data.zarr', mode='r')
alpha=pd.DataFrame(agent_data['alpha'])
net_trade=pd.DataFrame(agent_data['net_trade'])
k=pd.DataFrame(agent_data['wealth'])

fig=plt.figure()
plt.xlabel("α")
plt.ylabel("$k_{t}$")
#xlims=[0.8,OGdata.loc[TimeSample,"alpha"].max()]
xlims=[0.8,1.5]

ylims=[0,10]
#ylims=[0,OGdata.loc[TimeSample,"k_t"].max()]

fig.suptitle(f"k$_{{{'t'}}}$ v.s. α at Time Step {0}")



def animate3(i,alpha, k,net_trade, xlim,ylim):

    plt.clf()
    fig.suptitle(f"k$_{{{'t'}}}$ v.s. α at Time Step {i}")
    plt.xlabel("α")
    plt.ylabel("$k_{t}$")
    pointart=plt.scatter(alpha[i], k[i])
    plt.xlim(xlim)
    plt.ylim(ylim)
    snapshot=str(f"Net Traded:{net_trade[i].mean()}")
    fig.text(0.05, 0.021, snapshot,fontsize=10, bbox=dict(fc="white", ec="white"))





points=plt.scatter(alpha[0], k[0])
    #hue="i_a", 
    #palette=["b", "r","#7f7f7f"], 

plt.xlim(xlims)
plt.ylim(ylims)

anim3 = animation.FuncAnimation(fig, animate3, frames=k.shape[1], interval=2000, blit=False, fargs=(alpha,k,net_trade,xlims,ylims,)) 



plt.show()
'''

'\n# try to see something happening\nimport zarr\nimport pandas as pd\nimport matplotlib.pyplot as plt\nimport matplotlib.animation as animation\n\n\n\nagent_data = zarr.open(\'/Users/victoria/Documents/Scripts/Python/DGL-PTM/DGL_testing/agent_data.zarr\', mode=\'r\')\nalpha=pd.DataFrame(agent_data[\'alpha\'])\nnet_trade=pd.DataFrame(agent_data[\'net_trade\'])\nk=pd.DataFrame(agent_data[\'wealth\'])\n\nfig=plt.figure()\nplt.xlabel("α")\nplt.ylabel("$k_{t}$")\n#xlims=[0.8,OGdata.loc[TimeSample,"alpha"].max()]\nxlims=[0.8,1.5]\n\nylims=[0,10]\n#ylims=[0,OGdata.loc[TimeSample,"k_t"].max()]\n\nfig.suptitle(f"k$_{{{\'t\'}}}$ v.s. α at Time Step {0}")\n\n\n\ndef animate3(i,alpha, k,net_trade, xlim,ylim):\n\n    plt.clf()\n    fig.suptitle(f"k$_{{{\'t\'}}}$ v.s. α at Time Step {i}")\n    plt.xlabel("α")\n    plt.ylabel("$k_{t}$")\n    pointart=plt.scatter(alpha[i], k[i])\n    plt.xlim(xlim)\n    plt.ylim(ylim)\n    snapshot=str(f"Net Traded:{net_trade[i].mean()}")\n    fig.text(0.05, 0.021, s